# Generative models

In discriminative models like linear and logistic regression, we modeled the posterior $\mathbb{P}\{Y=k|X\}$ directly. In generative models, however, we turn to modeling the joint distribution 

$$\mathbb{P}\{X, Y\}=\mathbb{P}\{X|Y=k\}\mathbb{P}\{Y=k\}$$

By modeling the distribution $\mathbb{P}\{X|Y=k\}$ and $\mathbb{P}\{Y=k\}$. Once we've model these, we can recontruct the posterior using Bayes theorem

$$\mathbb{P}\{Y=k|X\}=\frac{\mathbb{P}\{X|Y=k\}\mathbb{P}\{Y=k\}}{\sum_{k} \mathbb{P}\{X|Y=k\}\mathbb{P}\{Y=k\}}$$